# Network model of epidemic

The reported study was funded by RFBR and CNPq, FASIE, DBT, DST, MOST, NSFC, SAMRC  according to the research project No 20-51-80002

In [1]:
import networkx as nx
import numpy as np
import torch
import matplotlib.pyplot as plt

In [2]:
POPULATION   = 10000
p_exposed    = int(POPULATION * 0.1)
p_infectious = int(POPULATION * 0.05)
p_removed    = 0
T = 75
END_OF_TIMES = 1000000

infectious_lambda = 1/0.186 # распределение вейбула αˆ= and λˆ=
infectious_alpha = 1.24

removed_lambda = 16.5 # mean = 14.46 
removed_alpha = 2.5   # sd = 6

death_rate = 0.03
infectious_rate = 0.0258

POPULATION, p_exposed, p_infectious, p_removed

(10000, 1000, 500, 0)

In [3]:
G = nx.watts_strogatz_graph(POPULATION, 13, 0.2) #4

In [4]:
# nx.draw(G)

In [5]:
nx.adjacency_matrix(G)
A = nx.adjacency_matrix(G)

In [6]:
AM = torch.tensor(A.todense())

In [7]:
state = torch.zeros(POPULATION)

In [8]:
# 0 - S: The susceptible individuals. When a susceptible and an infectious individual come into "infectious contact", the susceptible individual contracts the disease and transitions to the infectious compartment.
# 1 - E: Exposed. For many important infections, there is a significant incubation period during which individuals have been infected but are not yet infectious themselves. During this period the individual is in compartment E (for exposed).
# 2 - I: The infectious individuals. These are individuals who have been infected and are capable of infecting susceptible individuals.
# 3 - R: The removed (and immune) or deceased individuals

In [9]:
state[0:p_exposed] = 1

In [10]:
state[p_exposed:p_exposed+p_infectious]=2

In [11]:
state = state[torch.randperm(state.nelement())]

In [12]:
tau = torch.full([POPULATION], END_OF_TIMES)

In [13]:
ei_distr = torch.distributions.Weibull(torch.tensor([infectious_lambda]), torch.tensor([infectious_alpha]))
ir_distr = torch.distributions.Weibull(torch.tensor([removed_lambda]), torch.tensor([removed_alpha]))

In [14]:
for t in range(T):
    print(t)
    idx = (state == 1) & (tau == END_OF_TIMES)
    tau[idx] = torch.reshape(ei_distr.sample(tau[idx].size()).long(),(-1,)) + t

    idx = (state == 2) & (tau == END_OF_TIMES)
    tau[idx] = torch.reshape(ir_distr.sample(tau[idx].size()).long(),(-1,)) + t
    # print(tau)
    
    # Susceptible -> Exposed
    p = 1-(1-infectious_rate)**torch.sum((AM * (state==1)), dim=1)
    x = torch.rand(len(AM))
    idx = (x <= p) & (state==0)
    state[idx] = 1
    tau[idx] = END_OF_TIMES
    
    p = 1-(1-(infectious_rate/8))**torch.sum((AM * (state==2)), dim=1)
    x = torch.rand(len(AM))
    idx = (x <= p) & (state==0)
    state[idx] = 1
    tau[idx] = END_OF_TIMES
    
    # Exposed -> Infectious
    idx = (state == 1) & (tau == t)
    state[idx] = 2
    tau[idx] = END_OF_TIMES

    # Infectious -> Removed
    idx = (state == 2) & (tau == t)
    state[idx] = 3
    tau[idx] = END_OF_TIMES
    print(int(sum(state==0)),int(sum(state==1)),int(sum(state==2)),int(sum(state==3)))

0
8208 1180 611 1
1
7888 1321 784 7
2
7573 1444 971 12
3
7242 1524 1211 23
4
6911 1605 1448 36
5
6585 1675 1689 51
6
6264 1686 1976 74
7
5941 1707 2242 110
8
5636 1749 2443 172
9
5326 1763 2678 233
10
5066 1721 2906 307
11
4805 1657 3131 407
12
4556 1623 3291 530
13
4318 1569 3472 641
14
4103 1511 3593 793
15
3906 1431 3705 958
16
3712 1365 3784 1139
17
3528 1268 3847 1357
18
3359 1177 3901 1563
19
3210 1098 3923 1769
20
3085 995 3933 1987
21
2941 949 3885 2225
22
2819 905 3792 2484
23
2723 831 3680 2766
24
2640 758 3593 3009
25
2554 714 3487 3245
26
2471 664 3333 3532
27
2392 619 3202 3787
28
2310 579 3051 4060
29
2253 520 2935 4292
30
2207 464 2771 4558
31
2151 420 2645 4784
32
2106 389 2499 5006
33
2067 346 2366 5221
34
2022 326 2228 5424
35
1978 318 2087 5617
36
1930 321 1941 5808
37
1904 284 1816 5996
38
1874 256 1709 6161
39
1862 219 1587 6332
40
1841 189 1474 6496
41
1827 153 1406 6614
42
1810 139 1296 6755
43
1794 126 1187 6893
44
1779 120 1074 7027
45
1766 103 995 7136
46
1756